In [1]:
import matplotlib.pyplot as plt
import numpy as np
from   numpy.linalg import inv
import numpy.random as npr
from   pypolyagamma import PyPolyaGamma
from scipy.stats import multivariate_normal

from heapq import heappush, heappop
import os
import matplotlib.cm as cm
from tqdm import tqdm
import polars as pl

import geopandas as gpd

from shapely.geometry import Point, Polygon
from shapely import wkt

import pandas as pd
import statsmodels.api as sm
import scipy
import scipy.optimize as opt
import seaborn as sns

from pyproj import Transformer

from bayesian_statistics.utils_2 import *
import japanize_matplotlib

from keplergl import KeplerGl

from typing import Tuple

In [ ]:
data_dir = "/home/ohta/dev/bayesian_statistics/data"

obsedian_path = "11_gdf_obsedian.csv"
sites_path = "11_gdf_sites.csv"
elevation_path = "gdf_elevation_with_costs.csv"

df_obsedian = pl.read_csv(os.path.join(data_dir, obsedian_path))
df_sites = pl.read_csv(os.path.join(data_dir, sites_path))
df_elevation = pl.read_csv(os.path.join(data_dir, elevation_path))

In [7]:
def plot_grid_ratios(
    lon_mesh: np.ndarray,
    lat_mesh: np.ndarray,
    ratio_mesh: np.ndarray,
    df: pl.DataFrame,
    target_period: int,
    target_origin: str,
    radius_km: float,
    figsize: tuple = (12, 8)
) -> None:
    """
    グリッド点での比率を可視化（0-1の範囲に固定）
    """
    fig, ax = plt.subplots(figsize=figsize)
    
    # グリッド点での比率をプロット（0-1の範囲に固定）
    contour = ax.contourf(
        lon_mesh, lat_mesh, ratio_mesh,
        levels=np.linspace(0, 1, 21),  # 0から1までを20等分
        cmap='Blues',
        alpha=0.7,
        vmin=0,  # 最小値を0に固定
        vmax=1   # 最大値を1に固定
    )
    
    # 元のデータ点も表示
    unique_sites = df.unique(subset=['遺跡ID'])
    ax.scatter(
        unique_sites['経度'],
        unique_sites['緯度'],
        c='black',
        alpha=0.2,
        s=10
    )
    
    # タイトルと軸ラベル
    time_period_name = {
        0: "早期・早々期", 1: "前期", 2: "中期", 
        3: "後期", 4: "晩期"
    }
    ax.set_title(f'Obsidian Ratio Distribution (Grid-based)\n'
              f'Period: {time_period_name[target_period]}, '
              f'Origin: {target_origin}, Radius: {radius_km}km')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    # カラーバー（0-1の範囲に固定）
    plt.colorbar(contour, ax=ax, label='Ratio', ticks=np.linspace(0, 1, 6))
    
    return fig, ax


def plot_obsidian_distribution(
    df: pl.DataFrame,
    df_elevation: pl.DataFrame,
    lon_mesh: np.ndarray,
    lat_mesh: np.ndarray,
    ratio_mesh: np.ndarray,
    plot_df: pl.DataFrame,
    sigma: int,
    target_period: int,
    target_origin: str,
    grid_size: int,
    figsize: tuple = (8, 6)
) -> None:
    

    print("plotting...")
    # プロット
    plot_grid_ratios(
        lon_mesh, lat_mesh, ratio_mesh,
        df, target_period, target_origin,
        sigma, figsize
    )
    
    plt.scatter(
        plot_df["経度"], 
        plot_df["緯度"], 
        c=plot_df["比率"], 
        cmap="Blues", 
        edgecolors="white", 
        linewidths=0.5,
        vmin=0,
        vmax=1 
    )
    plt.show()